In [68]:
import numpy as numpy
import pandas as pd
import ast
import nltk
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")
movies=pd.merge(movies,credits,on="title")
movies= movies[["id","title","overview","genres","keywords","cast","crew"]]
movies.dropna(inplace=True)

def convert(obj):
  l=[]
  for i in ast.literal_eval(obj):
    l.append(i["name"])
  return l

def convert2(obj):
  l=[]
  counter=0
  for i in ast.literal_eval(obj):
    if counter<3:
      l.append(i["name"])
      counter+=1
    else :
      break
  return l


def convert3(obj):
  l=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      l.append(i["name"])
      break
  return l

       
        

movies.genres=movies.genres.apply(convert)
movies.keywords=movies.keywords.apply(convert)
movies.cast=movies.cast.apply(convert2)
movies.crew=movies.crew.apply(convert3)


movies.overview=movies.overview.apply(lambda x: x.split())

movies.cast=movies.cast.apply(lambda x: [i.replace(" ","") for i in x])
movies.crew=movies.crew.apply(lambda x: [i.replace(" ","") for i in x])
movies.genres=movies.genres.apply(lambda x: [i.replace(" ","") for i in x])
movies.keywords=movies.keywords.apply(lambda x: [i.replace(" ","") for i in x])

movies['tags']=movies["overview"]+movies["genres"]+movies["keywords"]+movies["cast"]+movies["crew"]

movies.drop(columns=["overview","genres","keywords","cast","crew"],inplace=True)

movies['tags']=movies["tags"].apply(lambda x: " ".join(x))

movies['tags']=movies["tags"].apply(lambda x: x.lower())


In [78]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem(text):
  l=[]
  for i in text.split():
    l.append(ps.stem(i))
  return " ".join(l)
movies['tags']=movies['tags'].apply(stem)

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(movies['tags']).toarray()
feature_names=cv.get_feature_names_out()


(4806, 5000)

In [85]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)
similarity.shape

(4806, 4806)

In [100]:
def recommend(movie):
  index=movies[movies['title']==movie].index[0]
  movies_list=sorted(list(enumerate(similarity[index])),reverse=True,key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(movies.iloc[i[0]].title)

In [107]:
import pickle
pickle.dump(movies.to_dict(),open('movies_dict','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))
